In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%autoreload # When utils.py is updated
from utils_unet_resunet import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from model.models import Model_3
import numpy as np
root_path = 'imgs/' 

In [3]:
# Define data type (L8-Landsat8, S2-Sentinel2, S1-Sentinel1)


sent2_2018_1 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2018_10m_b2348.tif').astype('float32')
sent2_2018_2 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2018_20m_b5678a1112.tif').astype('float32')

# Resize bands of 20m
sent2_2018_2 = resize_image(sent2_2018_2.copy(), sent2_2018_1.shape[0], sent2_2018_1.shape[1])
sent2_2018 = np.concatenate((sent2_2018_1, sent2_2018_2), axis=-1)
#sent2_2018 = sent2_2018_1.copy()
del sent2_2018_1, sent2_2018_2

sent2_2019_1 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2019_10m_b2348.tif').astype('float32')
sent2_2019_2 = load_tif_image(root_path+'New_Images/Sentinel2/'+'2019_20m_b5678a1112.tif').astype('float32')   

# Resize bands of 20m
sent2_2019_2 = resize_image(sent2_2019_2.copy(), sent2_2019_1.shape[0], sent2_2019_1.shape[1])
sent2_2019 = np.concatenate((sent2_2019_1, sent2_2019_2), axis=-1)
#sent2_2019 = sent2_2019_1.copy()
del sent2_2019_1, sent2_2019_2

# Filter outliers
sent2_2018 = filter_outliers(sent2_2018.copy()) 
sent2_2019 = filter_outliers(sent2_2019.copy()) 

opt_image_stack = np.concatenate((sent2_2018, sent2_2019), axis=-1)


# Load images
sar_2018_vh = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vh_2018.tif').astype('float32'), axis = -1)
sar_2018_vv = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vv_2018.tif').astype('float32'), axis = -1)
sar_2019_vh = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vh_2019.tif').astype('float32'), axis = -1)
sar_2019_vv = np.expand_dims(load_SAR_image(root_path+'New_Images/Sentinel1/'+'cut_sent1_vv_2019.tif').astype('float32'), axis = -1)

sar_2018 = np.concatenate((sar_2018_vh, sar_2018_vv), axis=-1)
sar_2019 = np.concatenate((sar_2019_vh, sar_2019_vv), axis=-1)
del sar_2018_vh, sar_2018_vv, sar_2019_vh, sar_2019_vv

# Filter outliers
sar_2018 = filter_outliers(sar_2018.copy()) 
sar_2019 = filter_outliers(sar_2019.copy()) 

sar_image_stack = np.concatenate((sar_2018, sar_2019), axis=-1)
del sar_2018, sar_2019
del sent2_2018, sent2_2019


#masking
final_mask1 = np.load(root_path+'New_Images/ref/'+'labels.npy')
lim_x = 10000
lim_y = 7000
opt_image_stack = opt_image_stack[:lim_x, :lim_y, :]
sar_image_stack = sar_image_stack[:lim_x, :lim_y, :]
final_mask1 = final_mask1[:lim_x, :lim_y]

#normalizing
type_norm = 1
opt_image_array = normalization(opt_image_stack.copy(), type_norm)
del opt_image_stack
print(np.min(opt_image_array), np.max(opt_image_array))
sar_image_array = normalization(sar_image_stack.copy(), type_norm)
del sar_image_stack
print(np.min(sar_image_array), np.max(sar_image_array))

#concatenate SAR and OPT
image_stack = np.concatenate((opt_image_array, sar_image_array), axis=-1)
print('Image stack:', image_stack.shape)


np.save(root_path+'New_Images/opt_stack.npy', opt_image_array)
np.save(root_path+'New_Images/sar_stack.npy', sar_image_array)
np.save(root_path+'New_Images/fus_stack.npy', image_stack)
np.save(root_path+'New_Images/final_mask1.npy', final_mask1)

del image_stack

# load references     
# Load current reference 
#ref_2019 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_2019.tif').astype('float32') # actual 2019
# Load past references
#past_ref = np.load(root_path+'New_Images/References/past_ref_and_clouds.npy').astype('float32')
#past_ref1 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_1988_2007.tif').astype('float32') # 1988_2007
#past_ref2 = load_tif_image(root_path+'New_Images/References/res_10m/r10m_def_2008_2018.tif').astype('float32') # 2008_2018
#clouds_2018 = load_tif_image(root_path+'New_Images/References/cut_b10_2018.tif').astype('float32')
#clouds_2018 = resize_image(np.expand_dims(clouds_2018.copy(), axis = -1), ref_2019.shape[0], ref_2019.shape[1])
#clouds_2018 = binary_mask_cloud(clouds_2018.copy(), 50)
#clouds_2019 = load_tif_image(root_path+'New_Images/References/cut_b10_2019.tif').astype('float32') 
#clouds_2019 = resize_image(np.expand_dims(clouds_2019.copy(), axis = -1), ref_2019.shape[0], ref_2019.shape[1])
#clouds_2019 = binary_mask_cloud(clouds_2019.copy(), 50)

imgs/New_Images/Sentinel2/2018_10m_b2348.tif
imgs/New_Images/Sentinel2/2018_20m_b5678a1112.tif
imgs/New_Images/Sentinel2/2019_10m_b2348.tif
imgs/New_Images/Sentinel2/2019_20m_b5678a1112.tif
imgs/New_Images/Sentinel1/cut_sent1_vh_2018.tif
imgs/New_Images/Sentinel1/cut_sent1_vv_2018.tif
imgs/New_Images/Sentinel1/cut_sent1_vh_2019.tif
imgs/New_Images/Sentinel1/cut_sent1_vv_2019.tif
-5.969623 5.626766
-1.3673552 6.424306
Image stack: (10000, 7000, 24)


In [4]:
# Print pertengate of each class (whole image)
print('Total no-deforestaion class is {}'.format(len(final_mask1[final_mask1==0])))
print('Total deforestaion class is {}'.format(len(final_mask1[final_mask1==1])))
print('Total past deforestaion class is {}'.format(len(final_mask1[final_mask1==2])))
print('Percentage of deforestaion class is {:.2f}'.format((len(final_mask1[final_mask1==1])*100)/len(final_mask1[final_mask1==0])))

Total no-deforestaion class is 36326397
Total deforestaion class is 1048775
Total past deforestaion class is 32624828
Percentage of deforestaion class is 2.89
